In [1]:
import numpy as np
import cv2
import csv
from pynq import allocate
from pynq import Overlay

ol = Overlay('bitstream/final_decr_v1_8.bit') 
ol?

In [3]:
#loading the pattern from .csv and choosing the 1

patterns = []  # each item is a list of 0/1 for that digit (digit 1-9)

with open("NEST_output/normalized_matrix.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)  # Load all rows into memory

    if rows:
        pattern_length = len(rows[0])  # Get the number of bits per pattern
#         patterns.append([0] * pattern_length)  # Add the 0th pattern (all zeros)

        for row in rows:
            pattern = [int(float(bit.strip())) for bit in row]
            patterns.append(pattern)
            
print(len(patterns[0]))
print(patterns)

61
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1

In [4]:
def pad_and_chunk_pattern(pattern):
    padded = pattern + [0] * (64 - len(pattern))  # add 3 zeros
    bitstring = "".join(str(bit) for bit in padded[::-1])  # reverse the bit list
    chunk_high = int(bitstring[32:64], 2)  # bits 32 to 63
    chunk_low =  int(bitstring[0:32], 2)  # bits 0 to 31
    return chunk_high, chunk_low

packed_chunks = []  # list of (high, low) tuples

for pattern in patterns:
    high, low = pad_and_chunk_pattern(pattern)
    packed_chunks.append((high, low))
    
print(packed_chunks)

[(0, 0), (0, 134217728), (0, 134217856), (2147483648, 134225920), (67108864, 268501024), (8388608, 134480385), (269484032, 135270416), (33816576, 135282817), (545390592, 136348168), (69238784, 138479649)]


In [6]:
#image

img_path = "encrypted_images/encrypt_7.png"
# Load image
img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)  # Preserve alpha channel
if img is None:
    print(f"Skipping {img_file}: Unable to read")
    exit()
    
# Get image details
height, width = img.shape[:2]
channels = img.shape[2]
dtype = img.dtype

# Print image details
print(f"Image Shape: {img.shape}")
print(f"Image Width: {width}, Height: {height}")
print(f"Number of Channels: {channels}")
print(f"Data Type: {dtype}")  

Image Shape: (1080, 1920, 4)
Image Width: 1920, Height: 1080
Number of Channels: 4
Data Type: uint8


In [7]:
num_pixels = height * width
num_entries = num_pixels
num_output = num_pixels//3
print(num_pixels)
print(num_output)

2073600
691200


In [8]:
# Allocate DMA buffer for PL communication (BGRA format, 32-bit per pixel)
input_buffer = allocate(shape=(num_entries + 24,), dtype=np.uint32) #24 = 10*2 + 4
output_buffer = allocate(shape=(num_output,), dtype=np.uint32)

bgra = img.reshape(-1, 4).astype(np.uint32)
input_buffer[24:] = (bgra[:, 0] << 24) | (bgra[:, 1] << 16) | (bgra[:, 2] << 8) | bgra[:, 3]
print("Data embedded successfully!")

Data embedded successfully!


In [9]:
print(img[0,0])
print(input_buffer)
print(output_buffer)
print(input_buffer.dtype)
# test_buffer = allocate(shape=(2,), dtype=np.uint32)
# test_buffer[0] = -99999
# print(test_buffer)

[  0   0   0 252]
[        0         0         0 ...     12540 336868604      4348]
[0 0 0 ... 0 0 0]
uint32


In [10]:
#writing initial 24 values
for i, chunk in enumerate(packed_chunks):
    input_buffer[2 * i] = chunk[0]
    input_buffer[2 * i + 1] = chunk[1]

key = np.load("key.npy")
print(key,key.dtype)

input_buffer[20] = key[1]
input_buffer[21] = key[0]
input_buffer[22] = 0
input_buffer[23] = 0

print(input_buffer)
print(num_entries)
print(len(input_buffer))

[       116 1395724288] uint32
[        0         0         0 ...     12540 336868604      4348]
2073600
2073624


In [11]:
dma = ol.axi_dma_0

In [12]:
#FLUSH & INVALIDATE BUFFERS
input_buffer.flush()         # Ensure data is written to memory before DMA reads it
output_buffer.invalidate()   # Ensure CPU reads fresh data after DMA writes it


In [17]:
# dma.sendchannel.stop()
# dma.recvchannel.stop()

# dma.register_map.MM2S_DMACR.Reset = 1  # Reset MM2S (Memory to Stream)
# dma.register_map.S2MM_DMACR.Reset = 1  # Reset S2MM (Stream to Memory)


# while dma.register_map.MM2S_DMACR.Reset:
#     pass  
# while dma.register_map.S2MM_DMACR.Reset:
#     pass

# dma.sendchannel.start()
# dma.recvchannel.start()

# print("DMA S2MM Status:", hex(dma.register_map.S2MM_DMASR))
# print("DMA MM2S Status:", hex(dma.register_map.MM2S_DMASR))
# print("Send Channel Running:", dma.sendchannel.running)
# print("Receive Channel Running:", dma.recvchannel.running)

In [13]:
# dma.sendchannel.transfer(input_buffer)
dma.sendchannel.transfer(input_buffer)
dma.recvchannel.transfer(output_buffer)
print("Send Channel Idle:", dma.sendchannel.idle)
print("Receive Channel Idle:", dma.recvchannel.idle)
print("MM2S Transfer Length:", hex(dma.register_map.MM2S_LENGTH))
print("DMA S2MM Status:", hex(dma.register_map.S2MM_DMASR))
print("DMA MM2S Status:", hex(dma.register_map.MM2S_DMASR))

dma.sendchannel.wait()
dma.recvchannel.wait()

Send Channel Idle: False
Receive Channel Idle: False
MM2S Transfer Length: 0x7e9060
DMA S2MM Status: 0x1002
DMA MM2S Status: 0x1002


In [14]:
print(input_buffer)
print(output_buffer)
print(output_buffer.dtype)

[        0         0         0 ...     12540 336868604      4348]
[0 0 0 ... 0 0 0]
uint32


In [15]:
print(img[0,:18,:])
print(input_buffer[:24])
print(output_buffer[:24])

[[  0   0   0 252]
 [  4   4   4 252]
 [ 12   8   4 252]
 [  8   0   0 252]
 [ 12   0   0 252]
 [ 16   4   0 252]
 [  8   4   0 252]
 [ 12   4   0 252]
 [ 12   4   0 252]
 [ 12   4   0 252]
 [ 12   4   0 252]
 [ 12   4   0 252]
 [  8   4   0 252]
 [ 12   4   0 252]
 [ 12   4   0 252]
 [ 12   0   0 252]
 [ 12   4   0 252]
 [ 12   4   0 252]]
[         0          0          0  134217728          0  134217856
 2147483648  134225920   67108864  268501024    8388608  134480385
  269484032  135270416   33816576  135282817  545390592  136348168
   69238784  138479649 1395724288        116          0          0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [13]:
print("Send Channel Idle:", dma.sendchannel.idle)
print("Receive Channel Idle:", dma.recvchannel.idle)

Send Channel Idle: True
Receive Channel Idle: True


In [16]:
# Decrypted Sound
decrypted_sound = output_buffer.astype(np.int32).reshape(-1, 2)

np.save("decrypted_sound/decrypted_sound_7.npy",decrypted_sound)
print("Decrypted sound is saved.")

Decrypted sound is saved.


In [26]:
# print(output_buffer - expected_output)

In [27]:
# print(expected_output)

In [28]:
# print(output_buffer)

In [29]:
# print(max(flattened_data))

In [30]:
# print(min(flattened_data))

In [31]:
# print(len(output_buffer))